### A gentle intro to event driven systems

This tutorial introduces some basic concepts of the event driven architecture
implemented in the simulator. You can find more details about this paradigm
[here](https://en.wikipedia.org/wiki/Event-driven_architecture). In this
example we will implement to very basic agents, a cat and a dog, that basically
talk to each other using message passing.

In [1]:
from just.simulate.env import Env
from just.simulate.registrable import Registrable

All agents in the simulator should extend the class `Registrable`, providing
the agents with some handy functions for interacting with the environment.
This `cat` has a `name` and subscribes the handler `woof_handler` to `Woof`.
This basically means that, every time the cat hears a `Woof` it will trigger
that handler. This cat also has a `meow` function that sends a `Meow` to the
environment and attaches some data to it, its name in particular.

In [2]:
class Cat(Registrable):

    def __init__(self, name: str) -> None:
        super().__init__()
        self.name = name

    def subscribe(self):
        self.add_subscription(
            'Woof', self.woof_handler)

    def meow(self):
        print(f"{self.name}: Meow!")
        self.env.publish('Meow', {'name': self.name})

    def woof_handler(self, data: dict, **kwargs) -> None:
        print(f"{self.name}: {data['name']} has woofed at me!")
        print(f"{self.name}: I will run away!")

Similarly, a `dog` will interact with the environment, but in this case it
will react to `meow`. In order to do that, it will subscribe `meow_handler`
to `Meow`, i.e. every time it hears a `meow` it will trigger that handler.

In [3]:
class Dog(Registrable):

    def __init__(self, name: str) -> None:
        super().__init__()
        self.name = name

    def subscribe(self):
        self.add_subscription(
            'Meow', self.meow_handler)

    def meow_handler(self, data: dict, **kwargs) -> None:
        print(f"{self.name}: {data['name']} has meowed at me!")
        print(f"{self.name}: I will woof at him!")
        self.env.publish('Woof', {'name': self.name})

Next, we create an environment where we register our cat and dog. Registering
these two agents will allow them to interact in the environment.

In [4]:
env = Env()
cat = Cat('Grumpy')
dog = Dog('Pavlov')
env.register(cat)
env.register(dog)

The cat starts the interaction meowing.

In [5]:
cat.meow()

Grumpy: Meow!


This is a discrete event simulator so we will need to run it. If the agents
were implemented as threads, then the dog will have replied immediately. We
have chosen discrete events because it seemed easier to implement things on,
much easier to debug, and also more efficient and faster in execution.

In [6]:
list(env.run())

Pavlov: Grumpy has meowed at me!
Pavlov: I will woof at him!
Grumpy: Pavlov has woofed at me!
Grumpy: I will run away!


[('Meow', {'name': 'Grumpy'}), ('Woof', {'name': 'Pavlov'})]